In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import string
import random
import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#SOS_token = 0
EOS_token = 0

In [3]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [4]:
class Data:
    def __init__(self):
        self.token2index = {"#":0}
        self.token2count = {}
        self.index2token = {0:"#"}
        self.n_tokens = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for token in list(sentence):
            self.addtoken(token)

    def addtoken(self, token):
        if token not in self.token2index:
            self.token2index[token] = self.n_tokens
            self.token2count[token] = 1
            self.index2token[self.n_tokens] = token
            self.n_tokens += 1
        else:
            self.token2count[token] += 1

In [5]:
MAX_LENGTH = 50


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(list(p[1])) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]
    
def prepareData(input_data_path, num_samples=10000):
    with  open(input_data_path, 'r', encoding='utf-8') as f:
        input_lines = f.read().split('\n')  # 行ごとのリストに
    print("Counting tokens:")
    data = Data()
    target_lines = []
    for line in input_lines:
        input_tokens = list(line)
        target_tokens = input_tokens[1:]
        target_tokens.append("#")
        target_lines.append("".join(target_tokens))
        data.addSentence(line)
    min_samples = min(num_samples, min(len(input_lines)-1, len(target_lines)-1))
    pairs = [[i,t]for (i,t) in zip(input_lines[:min_samples],target_lines[:min_samples])]
    pairs.append(input_lines)
    pairs.append(target_lines)
    pairs = filterPairs(pairs)

    print("Counted tokens:")
    print('data:{}'.format(data.n_tokens))
    return data, pairs

In [6]:
from sklearn.model_selection import train_test_split

input_data_path = "./data/text/mai2000a_token.txt"
#input_data_path = "./data/text/mai2000a_normal.txt"
data, pairs = prepareData(input_data_path, 30000000)
train_pairs, val_pairs = train_test_split(pairs, train_size=0.8)
print("train:"+ str(len(train_pairs)))
print("val:"+ str(len(val_pairs)))
print(random.choice(train_pairs))

Counting tokens:
Counted tokens:
data:4164
train:157364
val:39341
['開けてみたらスカーフだった。', 'けてみたらスカーフだった。#']


In [7]:
class LSTMPredictor(nn.Module):
    def __init__(self, input_dim, embed_dim, hidden_dim):
        super(LSTMPredictor, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeds = nn.Embedding(input_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, input_dim)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
        self.hidden = self.initHidden()

    def forward(self, input, hidden):
        embeds = self.embeds(input)
        lstm_out, hidden = self.lstm(
            embeds.view(len(input), 1, -1), hidden)
        output = self.linear(lstm_out.view(len(input), -1))
        output = self.dropout(output)
        output = self.softmax(output)
        #self.hidden = hidden
        return output, hidden

    def initHidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())

In [8]:
def indexesFromSentence(data, sentence):
    return [data.token2index[token] for token in list(sentence)]

def tensorFromSentence(data, sentence):
    indexes = indexesFromSentence(data, sentence)
    #indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(data, pair[0])
    target_tensor = tensorFromSentence(data, pair[1])
    return (input_tensor, target_tensor)

In [9]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, model, optimizer, criterion, max_length=MAX_LENGTH):
    model.train()
    hidden = model.initHidden()
    optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    loss = 0

    for i in range(input_length):
        output, hidden = model(input_tensor[i], hidden)
        loss += criterion(output, target_tensor[i])
    loss.backward()

    optimizer.step()
    return loss.item() / target_length

In [10]:
def validation(input_tensor, target_tensor, model, criterion):
    model.eval()
    with torch.no_grad():
        hidden = model.initHidden()
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)
        loss = 0
        for i in range(input_length):
            output, hidden = model(input_tensor[i], hidden)
            loss += criterion(output, target_tensor[i]) 
    return loss.item() / target_length

In [11]:
def trainEpochs(model, epochs=1, print_every=10000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_train_losses = []
    plot_val_losses = []
    print_train_loss_total = 0  # Reset every print_every
    print_val_loss_total = 0  # Reset every print_every
    plot_train_loss_total = 0  # Reset every plot_every
    plot_val_loss_total = 0  # Reset every plot_every

    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    train_length = len(train_pairs)
    val_length = len(val_pairs)
    training_pairs = [tensorsFromPair(random.choice(train_pairs)) for i in range(train_length)]
    validation_pairs = [tensorsFromPair(random.choice(val_pairs)) for i in range(val_length)]
    for epoch in range(epochs):
        #Train
        for j, train_pair in enumerate(training_pairs):
            i = j+1
            input_tensor = train_pair[0]
            target_tensor = train_pair[1]
            loss = train(input_tensor, target_tensor, model, optimizer, criterion)
            print_train_loss_total += loss
            plot_train_loss_total += loss

            if i % print_every == 0:
                print_loss_avg = print_train_loss_total / print_every
                print_train_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, i / train_length),
                                             i, i / train_length * 100, print_loss_avg))

            if i % plot_every == 0:
                plot_loss_avg = plot_train_loss_total / plot_every
                plot_train_losses.append(plot_loss_avg)
                plot_train_loss_total = 0
        
        #Validation        
        for j, val_pair in enumerate(validation_pairs):
            i = j+1
            input_tensor = train_pair[0]
            target_tensor = train_pair[1]
            loss = train(input_tensor, target_tensor, model, optimizer, criterion)
            print_val_loss_total += loss
            plot_val_loss_total += loss

            if i % print_every == 0:
                print_loss_avg = print_val_loss_total / print_every
                print_val_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, i / val_length),
                                             i, i / val_length * 100, print_loss_avg))

            if i % plot_every == 0:
                plot_loss_avg = plot_val_loss_total / plot_every
                plot_val_losses.append(plot_loss_avg)
                plot_val_loss_total = 0
            

    showPlot(plot_train_losses, "./results/"+str(epochs)+"_train.png")
    showPlot(plot_val_losses, "./results/"+str(epochs)+"val.png")

In [12]:
def trainIters(model, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()        
    
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, model, optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses,"./results/tken.png")

In [13]:
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
import matplotlib.ticker as ticker
import numpy as np
plt.style.use('ggplot')

def showPlot(points, figure_path):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.savefig(figure_path)

In [18]:
def evaluate(model, sentence, max_length=MAX_LENGTH):
    model.eval()
    with torch.no_grad():
        input_tensor = tensorFromSentence(data, sentence)
        input_length = input_tensor.size()[0]
        hidden = model.initHidden()
        outputs = []
        for i in range(input_length):
            output, hidden = model(input_tensor[i], hidden)
            outputs.append(output)
        return outputs

In [15]:
def evaluateRandomly(model, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        outputs = evaluate(model, pair[0])
        input_tokens = list(pair[0])
        for j, output in enumerate(outputs):
            if len(input_tokens) <= j: 
                break
            topv_list, topi_list = output[0].topk(5)
            print(input_tokens[j] + ":")
            output_tokens = []
            for index in topi_list:
                output_tokens.append(data.index2token[index.item()])
                
            print(output_tokens)

In [16]:
hidden_size = 256
embedding_dim = 256
epochs = 1
model = LSTMPredictor(data.n_tokens, embedding_dim, hidden_size).to(device)
trainEpochs(model, epochs, print_every=1000)

1m 55s (- 302m 15s) (1000 0%) 5.9021
3m 11s (- 248m 23s) (2000 1%) 5.2466
4m 26s (- 228m 31s) (3000 1%) 5.0736
5m 42s (- 218m 47s) (4000 2%) 4.9456
6m 59s (- 212m 56s) (5000 3%) 4.7669
8m 15s (- 208m 14s) (6000 3%) 4.6753
9m 29s (- 203m 58s) (7000 4%) 4.6225
10m 43s (- 200m 20s) (8000 5%) 4.6305
11m 59s (- 197m 37s) (9000 5%) 4.5603
13m 14s (- 195m 7s) (10000 6%) 4.4826
14m 30s (- 193m 5s) (11000 6%) 4.4625
15m 46s (- 191m 2s) (12000 7%) 4.4132
16m 59s (- 188m 41s) (13000 8%) 4.3533
18m 15s (- 186m 59s) (14000 8%) 4.3829
19m 30s (- 185m 13s) (15000 9%) 4.3520
20m 45s (- 183m 23s) (16000 10%) 4.3275
21m 59s (- 181m 37s) (17000 10%) 4.2995
23m 17s (- 180m 17s) (18000 11%) 4.2776
24m 34s (- 178m 55s) (19000 12%) 4.2653
25m 49s (- 177m 24s) (20000 12%) 4.2286
27m 5s (- 175m 56s) (21000 13%) 4.1974
28m 20s (- 174m 24s) (22000 13%) 4.1662
29m 39s (- 173m 14s) (23000 14%) 4.2110
30m 56s (- 171m 54s) (24000 15%) 4.2083
32m 11s (- 170m 24s) (25000 15%) 4.1685
33m 27s (- 169m 5s) (26000 16%) 4.1

In [19]:
evaluateRandomly(model, 10)

outputs = evaluate(model, "だが「市場の論理」万能のグローバル化はどうか。")

> これからが人生の総仕上げのときだ。
こ:
['れ', 'こ', 'の', 'う', 'ん']
れ:
['ま', 'は', 'に', 'で', 'ら']
か:
['ら', 'に', 'は', '、', 'り']
ら:
['本', 'は', 'の', '、', '１']
が:
['本', '、', '「', 'ん', '１']
人:
['気', '々', '権', '間', '事']
生:
['に', '活', 'は', 'を', 'の']
の:
['本', '方', 'こ', '、', '子']
総:
['理', '選', '統', '額', '事']
仕:
['事', '上', 'は', '子', '組']
上:
['げ', 'は', 'が', 'に', '化']
げ:
['は', 'の', 'を', 'だ', '者']
の:
['本', '、', '収', '一', '方']
と:
['こ', 'み', 'は', '、', '思']
き:
['、', 'は', 'に', 'ど', '言']
だ:
['、', 'っ', '。', 'ろ', '社']
。:
['#', '本', 'そ', '私', '＝']
> 各自治体には最大限の努力をお願いしたい。
各:
['社', '国', '地', '種', '省']
自:
['分', '治', '社', 'は', '民']
治:
['体', '省', '区', '相', '療']
体:
['は', 'に', 'の', 'も', 'で']
に:
['収', 'は', '本', 'よ', 'も']
は:
['、', '本', 'ど', '「', '社']
最:
['本', '高', '後', '終', '低']
大:
['の', 'は', '手', '規', 'に']
限:
['界', '定', 'に', '度', '問']
の:
['収', '本', '高', '問', '改']
努:
['力', 'ー', '収', '車', 'ベ']
力:
['に', '者', '収', '不', 'は']
を:
['収', '本', '求', '受', '呼']
お:
['寄', 'ろ', 'り', '願', 'す']
願:
['う', 'え', 'い', 'く', 'ま']
い:
['す', '、', 'ま', 'さ', 'し']


In [ ]:
torch.save(model.state_dict(), "./weights/token_level_"+str(epochs)+".h5")